In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

### The data set has been taken from : https://www.kaggle.com/datasets/hruturajbabar/rue-and-fake-news

### Steps to be followed 
1) view the dataset and add a column for fake or not .

2) Merge the dataset into one and use this as a new df

3) visualize the data and take insights from your data and plot using Wordcount and more

4) merge the title and article and then remove the stop words 

5) tokenize the words using the nltk word tokenizer and use this as your dataset

6) Prepare a model for your data (LSTM) and train on it 

7) Do predictions for your model . 

In [ ]:
df_fake = pd.read_csv('../input/fake-news-prediction/Fake.csv')
df_true = pd.read_csv('../input/fake-news-prediction/True.csv')



In [ ]:
df_fake.head()

In [ ]:
df_true.head()

In [ ]:
print(len(df_true))
print(len(df_fake))

In [ ]:
df_fake['isfake'] = 1
df_true['isfake'] = 0
print(df_fake.columns)

In [ ]:
df_true.head()

In [ ]:
# concat the dataset 
combined_data = [df_true , df_fake]
df_concat = pd.concat(combined_data)
df_concat

In [ ]:
df_concat.reset_index(inplace = True)

In [ ]:
df_concat.drop(['index' , 'date'] ,axis = 1, inplace = True)
df_concat

In [ ]:
df_concat['merged'] = df_concat['title']+df_concat['text']
df_concat.head()

In [ ]:
df_concat['merged'][0]

In [ ]:
import string
string.punctuation
# df['clean_merged'] = [char for char in df['merged'] if char not in string.punctuation]



In [ ]:
n = len(df_concat)
n

In [ ]:
df_concat['clean_mergedddd'] = 0;

In [ ]:
n = len(df_concat)
for i in range(0,n):
    
    punc = []
#     print("the i value is " , i , " the len(punc) is " , len(punc))
    for char in df_concat['merged'][i]:
        if char not in string.punctuation :
            punc.append(char)
    df_concat['clean_mergedddd'][i] = ''.join(punc)
#     print("len punc " , len(punc))
print("printing the head")
df_concat

In [ ]:
# remove stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
# stop_words

In [ ]:
df_concat['cleaned'] = 0;

In [ ]:
for i in range(0 , len(df_concat)):
    sentence = ""
    for word in df_concat['clean_mergedddd'][i].split():
        if word not in stop_words:
            sentence = sentence +  " " + word
        
    df_concat['cleaned'][i] = sentence

    
print("print the head")
df_concat.head()

In [ ]:
df_concat.drop(['clean_mergedddd'] , axis = 1 ,inplace  = True)
df_concat.head()

In [ ]:
# ex = "I am a good boy and Playing , resting is goofd"
# ple = []
# sentence = ""
# for word in ex.split():
    
#     if word not in stop_words:
#         sentence = sentence + " " + word

# print(sentence)

In [ ]:
# Now we need to tokenize 
list_of_words = []

for i in range(0 , len(df_concat)):
    for j in df_concat['cleaned'][i].split():
        list_of_words.append(j)
        
total_words = len(list(set(list_of_words)))


In [ ]:
total_words

In [ ]:
len(list_of_words)
# great I have bit doubt about this

In [ ]:
df_concat.head()

In [ ]:
# tokenize the model 
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df_concat['cleaned'], df_concat['isfake'], test_size = 0.2)

In [ ]:
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, Input, LSTM, Conv1D, MaxPool1D, Bidirectional
from tensorflow.keras.models import Model


#### One of the most important step for NLP models 

In [ ]:
from nltk.tokenize import word_tokenize
tokenizer = Tokenizer(num_words = total_words)
tokenizer.fit_on_texts(x_train)
train_sequences = tokenizer.texts_to_sequences(x_train)
test_sequences = tokenizer.texts_to_sequences(x_test)

In [ ]:
print("The encoding for document\n",df_concat['cleaned'][0],"\n is : ",train_sequences[0])

In [ ]:
# Lets check something I was curious about ... 
# length of tokenizer and actualy sentence lenght

In [ ]:
cnt = 0;
for words in df_concat['cleaned'][0].split():
    cnt=cnt+1

print("cnt is ",cnt)
print("len of arr is " , len(train_sequences[0]))

In [ ]:
len(train_sequences)

In [ ]:
x_train

In [ ]:
maxlen = -1
for doc in df_concat.cleaned:
    tokens = nltk.word_tokenize(doc)
    if(maxlen<len(tokens)):
        maxlen = len(tokens)
print("The maximum number of words in any document is =", maxlen)

In [ ]:
# Add padding can either be maxlen = 4406 or smaller number maxlen = 40 seems to work well based on results
padded_train = pad_sequences(train_sequences,maxlen = 40, padding = 'post', truncating = 'post')
padded_test = pad_sequences(test_sequences,maxlen = 40, truncating = 'post') 

In [ ]:
import plotly.express as px
fig = px.histogram(x = [len(nltk.word_tokenize(x)) for x in df_concat.cleaned], nbins = 100)
fig.show()

In [ ]:
x_test

In [ ]:
y_test

In [ ]:
len(padded_train)
len(padded_train[0])

In [ ]:
len(padded_train[1])

In [ ]:
len(x_train)

In [ ]:
total_words

In [ ]:
model = Sequential()
model.add(Embedding(total_words , output_dim = 50))
model.add(Bidirectional(LSTM(50)))
model.add(Dense(50 , activation = 'relu'))
model.add(Dense(1 ,activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy' , optimizer = 'adam' , metrics = ['acc'])
model.summary()

In [ ]:
y_train = np.asarray(y_train)

In [ ]:
y_train

In [ ]:
model.fit(padded_train , y_train , validation_split  =0.2 ,epochs = 3 , batch_size = 64)

In [ ]:
pred = model.predict(padded_test)
prediction = []
for i in range(len(pred)):
    if pred[i].item() > 0.5:
        prediction.append(1)
    else:
        prediction.append(0)
    


In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(list(y_test), prediction)

print("Model Accuracy : ", accuracy)

# get the confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(list(y_test), prediction)
plt.figure(figsize = (25, 25))
sns.heatmap(cm, annot = True)

In [ ]:
category = { 0: 'Fake News', 1 : "Real News"}

In [ ]:
model_2 = Sequential()
model_2.add(Embedding(total_words , output_dim = 128))
model_2.add(Bidirectional(LSTM(128)))
model_2.add(Dense(128 , activation = 'relu'))
model_2.add(Dense(1 , activation = 'sigmoid'))

model_2.compile(loss = 'binary_crossentropy' , optimizer = 'adam' , metrics = ['acc'])
model_2.summary()

In [ ]:
model_2.fit(padded_train , y_train , validation_split  =0.2 ,epochs = 3 , batch_size = 64)